In [1]:
# Dash&Flask
import dash
from dash import html, dcc
# Custom libraries
import layouts
from layouts import images_page, text_page
import utilities
import callbacks
from callbacks import app
# Others
import warnings
warnings.filterwarnings('ignore') # disable warnings relateds to versions of tf

In [2]:
# debug
from importlib import reload
reload(layouts)
reload(utilities)

<module 'utilities' from '/Users/giuliacrocioni/Desktop/eScience/projects_2022/DIANNA/dianna/dashboard/utilities.py'>

In [3]:
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

@app.callback(dash.dependencies.Output('page-content', 'children'),
              [dash.dependencies.Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/apps/text':
         return text_page
    else:
         return images_page # home page

app.run_server(mode='external', port=8050)

Dash app running on http://127.0.0.1:8050/
